In [1]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F 
from torchvision import transforms as T,datasets,models
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from collections import OrderedDict
from tqdm import tqdm
from torch import nn, optim
from torch.autograd import Variable
from torchsummary import summary
from torch.nn import BatchNorm2d
from torch.nn import Conv2d
from torch.nn import LeakyReLU
from torch.nn import Sigmoid

In [2]:
def data_transforms(phase = None):
    
    if phase == 'train':

        data_T = T.Compose([
            
                T.Resize(size = (256,256)),
                T.RandomRotation(degrees = (-20,+20)),
                T.CenterCrop(size=224),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
    
    else:

        data_T = T.Compose([

                T.Resize(size = (224,224)),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
    return data_T

In [3]:
data_dir = "C:/Users/Anil/Downloads/archive/chest_xray"

trainset = datasets.ImageFolder(os.path.join(data_dir, 'train'),transform = data_transforms('train'))
testset = datasets.ImageFolder(os.path.join(data_dir, 'test'),transform = data_transforms('test'))
validset = datasets.ImageFolder(os.path.join(data_dir, 'val'),transform = data_transforms('val'))

In [4]:
class_names = trainset.classes

In [5]:
trainloader = DataLoader(trainset,batch_size = 64,shuffle = True)
validloader = DataLoader(validset,batch_size = 64,shuffle = True)
testloader = DataLoader(testset,batch_size = 64,shuffle = True)

images, labels = next(iter(trainloader))

In [6]:
for i, (images,labels) in enumerate(trainloader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

In [7]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(

            nn.Conv2d(3, 64, (4, 4), (2, 2), (1, 1), bias=True),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(64, 128, (4, 4), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(128, 256, (4, 4), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(256, 512, (4, 4), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(512, 1, (4, 4), (1, 1), (0, 0), bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.main(x)
        out = torch.flatten(out, 1)

        return out

In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
model = Discriminator()

optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.BCELoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [10]:
model = Discriminator()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [11]:
print(model)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (12): Sigmoid()
  )
)


In [12]:
summary(model, (images.shape[1], images.shape[2], images.shape[3]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,136
         LeakyReLU-2         [-1, 64, 112, 112]               0
            Conv2d-3          [-1, 128, 56, 56]         131,072
       BatchNorm2d-4          [-1, 128, 56, 56]             256
         LeakyReLU-5          [-1, 128, 56, 56]               0
            Conv2d-6          [-1, 256, 28, 28]         524,288
       BatchNorm2d-7          [-1, 256, 28, 28]             512
         LeakyReLU-8          [-1, 256, 28, 28]               0
            Conv2d-9          [-1, 512, 14, 14]       2,097,152
      BatchNorm2d-10          [-1, 512, 14, 14]           1,024
        LeakyReLU-11          [-1, 512, 14, 14]               0
           Conv2d-12            [-1, 1, 11, 11]           8,193
          Sigmoid-13            [-1, 1, 11, 11]               0
Total params: 2,765,633
Trainable param

In [13]:
Losses = []
for i in range(20): #I noticed very little change in the loss, so keeping the epochs low
    running_loss = 0
    for images, labels in trainloader:
        
        if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()

        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        
        optimizer.step()
        Losses.append(loss)
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(i+1, running_loss/len(trainloader)))

Epoch 1 - Training loss: 3.923244833946228
Epoch 2 - Training loss: 3.86826235492055
Epoch 3 - Training loss: 3.8562225888415083
Epoch 4 - Training loss: 3.8538341551292232
Epoch 5 - Training loss: 3.8491066025524603
Epoch 6 - Training loss: 3.856597720122919
Epoch 7 - Training loss: 3.853225652764483
Epoch 8 - Training loss: 3.8498688645479158
Epoch 9 - Training loss: 3.848143493256918
Epoch 10 - Training loss: 3.8458196826097444
Epoch 11 - Training loss: 3.8439349401287917
Epoch 12 - Training loss: 3.8462044872888703
Epoch 13 - Training loss: 3.847195180450998
Epoch 14 - Training loss: 3.8445214847239053
Epoch 15 - Training loss: 3.8434906645518976
Epoch 16 - Training loss: 3.8443543649301297
Epoch 17 - Training loss: 3.8438061533904655
Epoch 18 - Training loss: 3.8448136637850507
Epoch 19 - Training loss: 3.843256208954788
Epoch 20 - Training loss: 3.84276822427424


In [14]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    img = images[i].view(1, 3, 224, 224)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.cpu()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 624

Model Accuracy = 0.3766025641025641
